## Library

In [37]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import operator
from math import sqrt

from collections import Counter
# Lemmatize with POS Tag
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import linear_model


## Loading Data 

In [2]:
D = np.load('count_vector_matrix.npy') # load

In [3]:
D.shape

(9025, 19931)

In [4]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
ratings = pd.read_csv('../the-movies-dataset/ratings_equal_movies_metadata.csv')
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
test = pd.read_csv('../the-movies-dataset/ratings_test.csv')

In [5]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 26 columns):
adult                    9025 non-null bool
belongs_to_collection    1670 non-null object
budget                   9025 non-null int64
genres                   9025 non-null object
homepage                 1954 non-null object
id                       9025 non-null int64
imdb_id                  9025 non-null object
original_language        9025 non-null object
original_title           9025 non-null object
overview                 9013 non-null object
popularity               9025 non-null float64
poster_path              9022 non-null object
production_companies     9025 non-null object
production_countries     9025 non-null object
release_date             9025 non-null object
revenue                  9025 non-null float64
runtime                  9025 non-null float64
spoken_languages         9025 non-null object
status                   9023 non-null object
tagline             

In [26]:
N = len(smd)
N

9025

In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99810 entries, 0 to 99809
Data columns (total 6 columns):
userId        99810 non-null int64
id            99810 non-null float64
rating        99810 non-null float64
movieId       99810 non-null int64
timestamp     99810 non-null int64
movieIndex    99810 non-null int64
dtypes: float64(2), int64(4)
memory usage: 4.6 MB


In [8]:
ratings

,userId,id,rating,movieId,timestamp,movieIndex
0,0,152.0,2.5,1371,1260759135,1107
1,0,9909.0,2.5,31,1260759144,30
2,0,847.0,2.0,2193,1260759198,1739
3,0,9426.0,2.5,2455,1260759113,1958
4,0,6114.0,3.5,1339,1260759125,1079
...,...,...,...,...,...,...
99805,670,786.0,2.0,3897,1063503718,3108
99806,670,7443.0,4.0,3751,1065111939,2995
99807,670,1891.0,5.0,1196,1064890635,949
99808,670,279.0,4.0,1225,1065149143,977


In [9]:
train.shape

(79848, 6)

In [10]:
test.shape

(19962, 6)

In [11]:
n_users = len(ratings['userId'].unique().astype('int'))
n_items = len(ratings['id'].unique().astype('int'))
n_users

671

In [12]:
len(train[train['userId'] == 1])

56

In [13]:
len(test[test['userId'] == 1])

19

In [14]:
ratings_train = train[['userId','id','rating','movieIndex']].values
ratings_test = test[['userId','id','rating','movieIndex']].values

In [15]:
ratings_train.shape

(79848, 4)

In [16]:
# this fuction return movie_index in meta_data_dataset and ratings of user  
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0] # get all the value in user_id column
    # get list of user_id index in y which value equal input 
    # user_id
    ids = np.where(y == user_id)[0]
    # get list movie_id matching user_id 
    item_ids = rate_matrix[ids, 3]
    item_ids = [int(i) for i in item_ids]
    
    scores = rate_matrix[ids,2]
    return (item_ids, scores)

In [17]:
a, b = get_items_rated_by_user(ratings_train, 0)
for i in a :
    print(i)

2375
1958
1107
1083
1037
1079
856
830
1043
1704
1511
2920
1013
1811
1136


In [18]:
train[train['userId'] == 0]

,userId,id,rating,movieId,timestamp,movieIndex
0,0,36819.0,1.0,2968,1260759200,2375
1,0,9426.0,2.5,2455,1260759113,1958
2,0,152.0,2.5,1371,1260759135,1107
3,0,1598.0,2.0,1343,1260759131,1083
4,0,665.0,2.0,1287,1260759187,1037
5,0,6114.0,3.5,1339,1260759125,1079
6,0,819.0,3.0,1061,1260759182,856
7,0,11360.0,3.0,1029,1260759179,830
8,0,783.0,2.0,1293,1260759148,1043
9,0,8393.0,3.0,2150,1260759194,1704


In [19]:
D.shape[1]

19931

In [20]:
d = D.shape[1] # movie vector dimention
W = np.zeros((d, n_users)) 
b = np.zeros((1,n_users))

In [21]:
for i in range(n_users):
        idx, scores = get_items_rated_by_user(ratings_train, i)
        clf = Ridge(alpha=.1, fit_intercept = True)
        Xhat = D[idx,:]
        clf.fit(Xhat, scores)
        W[:, i] = clf.coef_
        b[0, i] = clf.intercept_

In [31]:
Yhat = D.dot(W) + b 
Yhat.shape

(9025, 671)

In [33]:
def get_list_movie_have_not_rated():
    user_rated, score = get_items_rated_by_user(rate_matrix,userId)
    result = []
    for i in range(movie_count):
         if i not in user_rated: result.append(i)
    return (userId, result)            

In [74]:
def get_pred_rating(rate_matrix, userId,movie_count):
    userId, list_not_rate =  get_list_user_have_not_rated(rate_matrix, userId,movie_count)
    result = {}
    for i in list_not_rate:
        result[i] = Yhat[i, userId]
    
    result = sorted(result.items(), key=operator.itemgetter(1),reverse=True)
    list_movie = []
    for i in result:
        list_movie.append(i[0])
    return list_movie

In [75]:
#user_rated, score = get_items_rated_by_user(ratings_train,0)
#pd.DataFrame(smd.iloc[user_rated])

In [78]:
result = get_pred_rating(ratings_train,0,N)
pd.DataFrame(smd.iloc[result[0:10]])

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description,year
4933,False,NaN,0,"['Action', 'Crime', 'Thriller']",http://www.mgm.com/#/our-titles/2024/To-Live-a...,9846,tt0090180,en,To Live and Die in L.A.,A fearless Secret Service agent will stop at n...,...,116.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A federal agent is dead. A killer is loose. An...,To Live and Die in L.A.,False,6.8,129.0,A fearless Secret Service agent will stop at n...,1985
118,False,NaN,50000000,"['Action', 'Thriller', 'Mystery', 'Romance']",NaN,11863,tt0113451,en,Jade,Someone does a nasty hatchet job on a San Fran...,...,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some fantasies go too far.,Jade,False,5.1,45.0,Someone does a nasty hatchet job on a San Fran...,1995
4505,False,NaN,55000000,"['Drama', 'Action', 'Thriller', 'Crime']",NaN,10632,tt0269347,en,The Hunted,"In the wilderness of British Columbia, two hun...",...,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some men should not be found.,The Hunted,False,6.0,194.0,"In the wilderness of British Columbia, two hun...",2003
3770,False,NaN,0,"['Horror', 'Action', 'Drama', 'Mystery', 'Thri...",NaN,27958,tt0080569,en,Cruising,A serial killer brutally slays and dismembers ...,...,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Al Pacino is Cruising for a killer.,Cruising,False,6.0,89.0,A serial killer brutally slays and dismembers ...,1980
5763,False,NaN,22000000,"['Action', 'Thriller', 'Adventure']",NaN,38985,tt0076740,en,Sorcerer,Four exiled international criminals on the run...,...,121.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,WANTED. Four men willing to drive a cargo of d...,Sorcerer,False,7.5,104.0,Four exiled international criminals on the run...,1977
8045,False,NaN,10000000,"['Crime', 'Drama', 'Thriller']",http://killerjoethemovie.com/,73567,tt1726669,en,Killer Joe,A cop (Matthew McConaughey) who moonlights as ...,...,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Totally Deep-Fried Texas Redneck Trailer Par...,Killer Joe,False,6.4,433.0,A cop (Matthew McConaughey) who moonlights as ...,2011
2388,False,NaN,0,"['Drama', 'Horror', 'Mystery', 'Thriller']",NaN,32043,tt0099710,en,The Guardian,The idyllic lives of Phil and Kate seem comple...,...,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Tonight, while the world is asleep... an ancie...",The Guardian,False,5.3,32.0,The idyllic lives of Phil and Kate seem comple...,1990
1555,False,"{'id': 12263, 'name': 'The Exorcist Collection...",8000000,"['Drama', 'Horror', 'Thriller']",http://theexorcist.warnerbros.com/,9552,tt0070047,en,The Exorcist,12-year-old Regan MacNeil begins to adapt an e...,...,122.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Something almost beyond comprehension is happe...,The Exorcist,False,7.5,2046.0,12-year-old Regan MacNeil begins to adapt an e...,1973
7484,False,NaN,0,"['Crime', 'Drama']",NaN,12219,tt0118528,en,12 Angry Men,During the trial of a man accused of his fathe...,...,117.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,12 Angry Men,False,7.5,59.0,During the trial of a man accused of his fathe...,1997
374,False,NaN,0,['Drama'],NaN,19819,tt0109305,en,Blue Chips,"Pete Bell, a college basketball coach is under...",...,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Blue Chips,False,5.7,41.0,"Pete Bell, a college basketball coach is under...",1994


In [ ]:
n = 0
np.set_printoptions(precision=2)
ids, scores = get_items_rated_by_user(ratings_test, n)
result = Yhat[ids, n]
result

In [ ]:
#test[test['userId'] == 0]

In [ ]:
print("ratings prediction of user ",n," :")
for i in range(len(result)):
    print("movie id : ",ids[i]," - ","predict ratings : ",result[i]," - ","true ratings : ",scores[i])

In [ ]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return sqrt(se/cnt)

In [ ]:
print('RMSE for test case:', evaluate(Yhat, ratings_train, W, b))
print('RMSE for test case:', evaluate(Yhat, ratings_test, W, b))

In [ ]:
np.save('count_pred_rating.npy', Yhat)